Import Packages

In [ ]:
import folium
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio import Affine
from rasterio.features import shapes
from shapely.geometry import shape, box, Polygon, LineString, MultiLineString
import numpy as np
from PIL import Image
from io import BytesIO
import base64
import os
import warnings

warnings.filterwarnings('ignore')

Load Abu Dhabi Shapefile (Admin Level 3)

In [2]:
# Load the shapefile using GeoPandas
data_folder = "../Data/UAE_Shapefile/gadm41_ARE_shp"
shapefile_name = "gadm41_ARE_3.shp"
shapefile_path = os.path.join(data_folder, shapefile_name)
gdf = gpd.read_file(shapefile_path)
print("Columns : ", gdf.columns.tolist())

Columns :  ['GID_3', 'GID_0', 'COUNTRY', 'GID_1', 'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2', 'NL_NAME_2', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3', 'geometry']


Filter for Abu Dhabi Region

In [3]:
# Abu Dhabi City
abu_dhabi_gdf = gdf[gdf["NAME_1"] == "Abu Dhabi"]
print(f"Shape: {abu_dhabi_gdf.shape}")
abu_dhabi_gdf.head(5)

Shape: (125, 17)


,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,ARE.1.1.2_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Abu Al Abyad,NA,NA,Municipality,Municipality,NA,NA,"MULTIPOLYGON (((53.87441 24.26483, 53.87717 24..."
1,ARE.1.1.3_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Abu Dhabi Island,NA,NA,Municipality,Municipality,NA,NA,"MULTIPOLYGON (((54.28496 24.4461, 54.28496 24...."
2,ARE.1.1.4_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Airport District,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.61171 24.45358, 54.61182 24.45385..."
3,ARE.1.1.5_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Al Falah,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.67202 24.40866, 54.67627 24.41914..."
4,ARE.1.1.6_1,ARE,United Arab Emirates,ARE.1_1,Abu Dhabi,إمارة أبوظبي,ARE.1.1_1,Abu Dhabi,أبو ظبي,Al Jubail Island,NA,NA,Municipality,Municipality,NA,NA,"POLYGON ((54.52723 24.51838, 54.52185 24.51998..."


Calculate the Midpoint Coordinates

In [4]:
# total_bounds → (minx, miny, maxx, maxy) for the entire GeoDataFrame
minx, miny, maxx, maxy = abu_dhabi_gdf.total_bounds

# x corresponds to longitude, y corresponds to latitude
sw_lat = miny  # southernmost lat
sw_lon = minx  # westernmost lon
ne_lat = maxy  # northernmost lat
ne_lon = maxx  # easternmost lon

print("Overall Bounding Box for KSA Polygons:")
print(f"  SW corner (lat, lon): ({sw_lat}, {sw_lon})")
print(f"  NE corner (lat, lon): ({ne_lat}, {ne_lon})")

# Calculate midpoint coordinates
mid_lat = (sw_lat + ne_lat) / 2
mid_lon = (sw_lon + ne_lon) / 2

Overall Bounding Box for KSA Polygons:
  SW corner (lat, lon): (22.631621401000018, 51.49797800000016)
  NE corner (lat, lon): (25.252622605000056, 56.018131256000174)


Load and Mask the TIF file -Doesn't cover all Abu Dhabi Regions

In [19]:
# Set the TIF file path
tif_path = "../Data/UAE_Shapefile/terrascope_download/WORLDCOVER/ESA_WORLDCOVER_10M_2021_V200/MAP/ESA_WorldCover_10m_2021_v200_N24E054_Map/ESA_WorldCover_10m_2021_v200_N24E054_Map.tif"

# Load it
with rasterio.open(tif_path) as src:
    # Reproject GeoDataFrame to match raster CRS
    abu_dhabi_gdf = abu_dhabi_gdf.to_crs(src.crs)

    # Mask the raster using the Abu Dhabi boundary
    out_image, out_transform = mask(src, abu_dhabi_gdf.geometry, crop=True)
    out_meta = src.meta.copy()

# ESA Code for built-up area is 50
built_up_mask = (out_image[0] == 50).astype(np.uint8) * 255  # Binary mask

# Convert the mask to an RGBA image (red color for built-up)
rgba = np.zeros((built_up_mask.shape[0], built_up_mask.shape[1], 4), dtype=np.uint8)
rgba[..., 0] = 255               # Red
rgba[..., 3] = built_up_mask     # Alpha channel: only show where built-up

# Convert to PIL image
image = Image.fromarray(rgba, mode='RGBA')

# Convert the image to base64 to embed in HTML
buffered = BytesIO()
image.save(buffered, format="PNG")
encoded = base64.b64encode(buffered.getvalue()).decode()

# Get bounds in lat/lon for folium image overlay
with rasterio.open(tif_path) as src:
    bounds = rasterio.transform.array_bounds(built_up_mask.shape[0], built_up_mask.shape[1], out_transform)
    bbox = rasterio.warp.transform_bounds(src.crs, 'EPSG:4326', *bounds)  # (minx, miny, maxx, maxy)

Create Folium Map

In [20]:
# Create a cartodb folium map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add Abu Dhabi boundary
folium.GeoJson(
    data=abu_dhabi_gdf.to_crs("EPSG:4326"),
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_3'],
        aliases=['District: '],
        localize=True
    )
).add_to(m)

# Add the built-up overlay
folium.raster_layers.ImageOverlay(
    image='data:image/png;base64,' + encoded,
    bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],  # [[south, west], [north, east]]
    opacity=0.6,
    name='ESA Built-up Areas'
).add_to(m)

# Add Layer control
folium.LayerControl().add_to(m)

Save the map

In [21]:
# Save map
output_map = '../Data/Maps/abu_dhabi_built_up_map.html'
m.save(output_map)
print(f"Map saved to {output_map}")

Map saved to ../Data/Maps/abu_dhabi_built_up_map.html


Combine all TIF files of UAE to cover all Abu Dhabi boundaries

In [ ]:
# Get the polygon
abu_dhabi_geom = abu_dhabi_gdf.unary_union

In [ ]:
# Set the TIF folder that cointain all TIF files
tif_folder = "../Data/ESA_TIF"
tif_files = [
    os.path.join(root, file)
    for root, dirs, files in os.walk(tif_folder)
    for file in files
    if file.lower().endswith("map.tif")
]

built_up_polygons = []

# Helper to check raster intersection
def raster_intersects(src, geom):
    raster_bounds = box(*src.bounds)
    return geom.intersects(raster_bounds)

# Downsample function
def downsample_raster(image, factor=2):
    return image[::factor, ::factor]

# Simplify polygons
def simplify_geometry(geom, tolerance=0.01):
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance, preserve_topology=True)
    return geom

for tif_path in tif_files:
    with rasterio.open(tif_path) as src:
        # Make sure the raster intersects Abu Dhabi
        if not raster_intersects(src, abu_dhabi_geom):
            continue

        try:
            out_image, out_transform = mask(src, [abu_dhabi_geom], crop=True)
            out_image = out_image[0]

            # Downsample
            factor = 4
            downsampled_image = downsample_raster(out_image, factor=factor)

            # Fix the transform scaling
            downsampled_transform = out_transform * Affine.scale(factor, factor)

            # Extract shapes
            for geom_val, val in shapes(
                    downsampled_image,
                    mask=(downsampled_image == 50),
                    transform=downsampled_transform
            ):
                if val == 50:
                    poly = shape(geom_val)

                    # Filter out tiny polygons
                    if poly.area < 1e-7:
                        continue

                    # Simplify geometry more aggressively
                    poly_simpl = simplify_geometry(poly, tolerance=0.01)
                    built_up_polygons.append(poly_simpl)

        except Exception as e:
            print(f"Failed to process {tif_path}: {e}")

# Convert to a GeoDataFrame
built_up_gdf = gpd.GeoDataFrame(geometry=built_up_polygons, crs="EPSG:4326")  # or the actual CRS if different

Build Folium map to display the Built-Up Areas

In [ ]:
# Build Folium map to display the Built-Up Areas
center = abu_dhabi_gdf.geometry.centroid.iloc[0]
m = folium.Map(location=[center.y, center.x], zoom_start=9, tiles='cartodbpositron')

# Add built-up polygons
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-Up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Add Abu Dhabi boundary
folium.GeoJson(
    abu_dhabi_gdf.__geo_interface__,
    name="Abu Dhabi Boundary",
    style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0}
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save("../Data/Maps/built_up_areas_map_lightweight.html")